In [1]:
# import transformers as tr
# tr.__version__

In [ ]:
pip install sentence-transformers

# BERT 임베딩을 활용하는 키워드 추출 모델


## 데이터 전처리

###1. 데이터 불러오기

In [3]:
import pandas as pd
from pandas import DataFrame
from pandas import Series
import numpy as np

In [4]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [5]:
df = pd.read_csv("/content/gdrive/MyDrive/DataScience/data/navershopping_airpod/review_airpod_encoded.csv")

# 평점 3점 행 삭제
idx = df[df['star']==3].index
df = df.drop(idx)
df.reset_index(drop=True, inplace=True)

In [6]:
df

,type,category,review,star
0,에어팟 1세대,음질,2세대로 배송 받았고 철가루랑 하얀색 실리콘케이스도 잘 받았습니다 역시 사용해보니 ...,5
1,에어팟 1세대,음질,"배송은 4시 이전 주문 당일 발송으로 다음날 바로 받았구요, 바로 정품 등록했는데 ...",5
2,에어팟 1세대,음질,아이폰과 아이패드를 둘 다 사용하고 있는데 아이폰에서 음악을 듣거나 영상을 보다 아...,5
3,에어팟 1세대,음질,다른 저렴한 곳도 있었습니다.\n\n그래도 제조일자 최신에 페이백으로 다시 받는 금...,4
4,에어팟 1세대,음질,택배도 하루일찍오고 하자 하나도 없고 깔끔했어요 뽁뽁이로 포장도 해주셔서 너무 감사...,5
...,...,...,...,...
3601,에어팟 2세대,착용감,생각 보다 착용감이 좋습니다. 즐거운 사운드를 느꼈습니다,5
3602,에어팟 2세대,착용감,우선 이전에 에어팟을 써본적은 없고 갤럭시 기어 아이콘 2018년 형을 썼는데 에어...,5
3603,에어팟 2세대,휴대성,"휴대하기 편하고, 애플 기기 사이의 호환성이 좋은 무선 이어폰이 필요해서 에어팟을 ...",4
3604,에어팟 2세대,휴대성,휴대하기도 좋고 음질도 좋은데 떨어져서 잊어 버릴까 불안 하네요~,4


In [7]:
df_ = pd.read_csv("/content/gdrive/MyDrive/DataScience/data/navershopping_airpod/sentimental_output.csv")

In [8]:
df_

,document,evaluation,label
0,2세대로 배송 받았고 철가루랑 하얀색 실리콘케이스도 잘 받았습니다 역시 사용해보니 ...,(긍정 확률 : 0.99) 긍정적인 리뷰입니다.,1
1,"배송은 4시 이전 주문 당일 발송으로 다음날 바로 받았구요, 바로 정품 등록했는데 ...",(긍정 확률 : 0.99) 긍정적인 리뷰입니다.,1
2,아이폰과 아이패드를 둘 다 사용하고 있는데 아이폰에서 음악을 듣거나 영상을 보다 아...,(긍정 확률 : 0.99) 긍정적인 리뷰입니다.,1
3,다른 저렴한 곳도 있었습니다.\n\n그래도 제조일자 최신에 페이백으로 다시 받는 금...,(긍정 확률 : 0.99) 긍정적인 리뷰입니다.,1
4,택배도 하루일찍오고 하자 하나도 없고 깔끔했어요 뽁뽁이로 포장도 해주셔서 너무 감사...,(긍정 확률 : 0.99) 긍정적인 리뷰입니다.,1
...,...,...,...
3601,생각 보다 착용감이 좋습니다. 즐거운 사운드를 느꼈습니다,(긍정 확률 : 0.99) 긍정적인 리뷰입니다.,1
3602,우선 이전에 에어팟을 써본적은 없고 갤럭시 기어 아이콘 2018년 형을 썼는데 에어...,(긍정 확률 : 0.99) 긍정적인 리뷰입니다.,1
3603,"휴대하기 편하고, 애플 기기 사이의 호환성이 좋은 무선 이어폰이 필요해서 에어팟을 ...",(긍정 확률 : 0.86) 긍정적인 리뷰입니다.,1
3604,휴대하기도 좋고 음질도 좋은데 떨어져서 잊어 버릴까 불안 하네요~,(긍정 확률 : 0.99) 긍정적인 리뷰입니다.,1


In [9]:
df = df.join(df_, how='left')
df.drop(['document', 'star'], axis=1, inplace=True)
df

,type,category,review,evaluation,label
0,에어팟 1세대,음질,2세대로 배송 받았고 철가루랑 하얀색 실리콘케이스도 잘 받았습니다 역시 사용해보니 ...,(긍정 확률 : 0.99) 긍정적인 리뷰입니다.,1
1,에어팟 1세대,음질,"배송은 4시 이전 주문 당일 발송으로 다음날 바로 받았구요, 바로 정품 등록했는데 ...",(긍정 확률 : 0.99) 긍정적인 리뷰입니다.,1
2,에어팟 1세대,음질,아이폰과 아이패드를 둘 다 사용하고 있는데 아이폰에서 음악을 듣거나 영상을 보다 아...,(긍정 확률 : 0.99) 긍정적인 리뷰입니다.,1
3,에어팟 1세대,음질,다른 저렴한 곳도 있었습니다.\n\n그래도 제조일자 최신에 페이백으로 다시 받는 금...,(긍정 확률 : 0.99) 긍정적인 리뷰입니다.,1
4,에어팟 1세대,음질,택배도 하루일찍오고 하자 하나도 없고 깔끔했어요 뽁뽁이로 포장도 해주셔서 너무 감사...,(긍정 확률 : 0.99) 긍정적인 리뷰입니다.,1
...,...,...,...,...,...
3601,에어팟 2세대,착용감,생각 보다 착용감이 좋습니다. 즐거운 사운드를 느꼈습니다,(긍정 확률 : 0.99) 긍정적인 리뷰입니다.,1
3602,에어팟 2세대,착용감,우선 이전에 에어팟을 써본적은 없고 갤럭시 기어 아이콘 2018년 형을 썼는데 에어...,(긍정 확률 : 0.99) 긍정적인 리뷰입니다.,1
3603,에어팟 2세대,휴대성,"휴대하기 편하고, 애플 기기 사이의 호환성이 좋은 무선 이어폰이 필요해서 에어팟을 ...",(긍정 확률 : 0.86) 긍정적인 리뷰입니다.,1
3604,에어팟 2세대,휴대성,휴대하기도 좋고 음질도 좋은데 떨어져서 잊어 버릴까 불안 하네요~,(긍정 확률 : 0.99) 긍정적인 리뷰입니다.,1


In [10]:
df['review'][0]

'2세대로 배송 받았고 철가루랑 하얀색 실리콘케이스도 잘 받았습니다 역시 사용해보니 왜 에어팟하는지 알겠더라구요 음질이 깨끗하고 좋고 톡톡 하면 다음곡으로 넘어가요 그리고 되게 귀가 안아프고 편해요~예쁘고 착용감이 좋아요>_<'

###2. 불용어 정리

In [11]:
# review 컬럼만 리스트로 저장
text=''
reviews=[]
for each_review in df['review']:
    reviews.append(each_review)

In [12]:
reviews

['2세대로 배송 받았고 철가루랑 하얀색 실리콘케이스도 잘 받았습니다 역시 사용해보니 왜 에어팟하는지 알겠더라구요 음질이 깨끗하고 좋고 톡톡 하면 다음곡으로 넘어가요 그리고 되게 귀가 안아프고 편해요~예쁘고 착용감이 좋아요>_<',
 '배송은 4시 이전 주문 당일 발송으로 다음날 바로 받았구요, 바로 정품 등록했는데 이상 없이 잘 됐어요!~연결도 잘되고, 음악 잠깐 들었는데 음질도 좋고 정말 편해요^^미리 구입해둔 케이스도 장착해서 바로 사용했어요~가격도 L.POINT랑 청구할인 받아서 직구보다 저렴하게 잘 구입했습니다.좋은 상품 감사합니다:)',
 '아이폰과 아이패드를 둘 다 사용하고 있는데 아이폰에서 음악을 듣거나 영상을 보다 아이패드로 바꿔서 들을때 불편해서 블루투스 이어폰을 검색하다 주변의 구매자들 모두 하나같이 최고다라고 이야기 한 애플의 에어팟을 구매하였습니다.\n\n할인쿠폰과 카드청구할인 혜택으로 저렴하게 구입할 수 있었고 배송도 공휴일이 포함되었음에도 2일만에 받을 수 있어 좋았습니다.\n\n제품은 박스를 에어캡으로 감싸고 다시 큰 박스에 넣어 안전하게 배송이 되었습니다. 제품도 2018년 5월 제조 최신 제품으로 받았습니다.\n\n받자마자 아이폰에 연결하고 사용해봤는데 . 대단하 만족합니다.',
 '다른 저렴한 곳도 있었습니다.\n\n그래도 제조일자 최신에 페이백으로 다시 받는 금액 생각해보니 얼추 괜찮다고 생각해서 구매했습니다.\n\n사용해보니 정말 편합니다. 음질도 나쁘지 않구요.\n귀에 꼽고 장시간 있으면 혹시 떨어졌나 하는 생각도 들구요 가끔씩 귀 한번 만집니다 ㅋㅋㅋ\n적응하면 신경 안쓸듯합니다.너무 불안하면 실리콘 줄 하나 구매해서 쓰려고 합니다',
 '택배도 하루일찍오고 하자 하나도 없고 깔끔했어요 뽁뽁이로 포장도 해주셔서 너무 감사했어요\n친구들 말만듣고 좋구나...생각만 했는데\n막상 사용하니 너무 좋아요 어제 노래들으면서 청소 했는데 삶의 질이 달라졌다는 말이 생각 나더라구요 ㅎㅎ 너무 마음에 들어요 그리고 차에서 선이 없으니

In [13]:
reviews[0]

'2세대로 배송 받았고 철가루랑 하얀색 실리콘케이스도 잘 받았습니다 역시 사용해보니 왜 에어팟하는지 알겠더라구요 음질이 깨끗하고 좋고 톡톡 하면 다음곡으로 넘어가요 그리고 되게 귀가 안아프고 편해요~예쁘고 착용감이 좋아요>_<'

In [14]:
#불용어 제거
import re
def clean_str(text):
    pattern = '([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)' # E-mail제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+' # URL제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '([ㄱ-ㅎㅏ-ㅣ]+)'  # 한글 자음, 모음 제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '<[^>]*>'         # HTML 태그 제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '[^\w\s]'         # 특수기호제거
    text = re.sub(pattern=pattern, repl='', string=text)
    return text   

review=[]
for i in reviews:  #문자가 들어있을때는 인덱스 사용하면 안됨!!!!
    a=clean_str(i)
    review.append(a)  #불용어제거한 review 저장

In [15]:
review[0]

'2세대로 배송 받았고 철가루랑 하얀색 실리콘케이스도 잘 받았습니다 역시 사용해보니 왜 에어팟하는지 알겠더라구요 음질이 깨끗하고 좋고 톡톡 하면 다음곡으로 넘어가요 그리고 되게 귀가 안아프고 편해요예쁘고 착용감이 좋아요_'

## 키워드 추출

### 1. 추출 문서

###2. 후보 키워드  
문서에서 후보 키워드 또는 핵심 문구 목록을 만들기. Scikit-Learns의  CountVectorizer를 사용하여 간단하게 키워드 목록으로 분리. 이를 통해 키워드의 길이를 지정하고이를 핵심 문구로 만들 수 있다. 또한 불용어를 빠르게 제거 할 수있는 좋은 방법이다.

In [16]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
# 키워드 길이 지정
n_gram_range = (1, 1)

In [17]:
# 리뷰하나씩 키워드로 분리(count-vectorizer)
a= len(review) # 불용어 제거한 리뷰

token_review_list=[]

for i in range(0,a): # 리뷰하나당 처리하기위해 for문 
    review_vectorized = CountVectorizer(ngram_range=n_gram_range).fit([review[i]])
    token_review = review_vectorized.get_feature_names()
    
    token_review_list.append(token_review) # 하나의 리스트를 만들어서 df에 추가해야함.
df['review_count']=token_review_list # 토큰 단위로 나누어진 리뷰저장

In [18]:
# 리뷰하나씩 키워드로 분리(tfidf-vectorizer)
token_review_list=[]

for i in range(0,a): # 리뷰하나당 처리하기위해 for문 
    review_vectorized = TfidfVectorizer(ngram_range=n_gram_range).fit([review[i]])
    token_review = review_vectorized.get_feature_names()
    
    token_review_list.append(token_review) # 하나의 리스트를 만들어서 df에 추가해야함.
df['review_tfidf']=token_review_list # 토큰 단위로 나누어진 리뷰저장

In [19]:
df

,type,category,review,evaluation,label,review_count,review_tfidf
0,에어팟 1세대,음질,2세대로 배송 받았고 철가루랑 하얀색 실리콘케이스도 잘 받았습니다 역시 사용해보니 ...,(긍정 확률 : 0.99) 긍정적인 리뷰입니다.,1,"[2세대로, 귀가, 그리고, 깨끗하고, 넘어가요, 다음곡으로, 되게, 받았고, 받았...","[2세대로, 귀가, 그리고, 깨끗하고, 넘어가요, 다음곡으로, 되게, 받았고, 받았..."
1,에어팟 1세대,음질,"배송은 4시 이전 주문 당일 발송으로 다음날 바로 받았구요, 바로 정품 등록했는데 ...",(긍정 확률 : 0.99) 긍정적인 리뷰입니다.,1,"[4시, lpoint랑, 감사합니다, 구입해둔, 구입했습니다좋은, 다음날, 당일, ...","[4시, lpoint랑, 감사합니다, 구입해둔, 구입했습니다좋은, 다음날, 당일, ..."
2,에어팟 1세대,음질,아이폰과 아이패드를 둘 다 사용하고 있는데 아이폰에서 음악을 듣거나 영상을 보다 아...,(긍정 확률 : 0.99) 긍정적인 리뷰입니다.,1,"[2018년, 2일만에, 5월, 감싸고, 검색하다, 공휴일이, 구매자들, 구매하였습...","[2018년, 2일만에, 5월, 감싸고, 검색하다, 공휴일이, 구매자들, 구매하였습..."
3,에어팟 1세대,음질,다른 저렴한 곳도 있었습니다.\n\n그래도 제조일자 최신에 페이백으로 다시 받는 금...,(긍정 확률 : 0.99) 긍정적인 리뷰입니다.,1,"[가끔씩, 곳도, 괜찮다고, 구매해서, 구매했습니다, 귀에, 그래도, 금액, 꼽고,...","[가끔씩, 곳도, 괜찮다고, 구매해서, 구매했습니다, 귀에, 그래도, 금액, 꼽고,..."
4,에어팟 1세대,음질,택배도 하루일찍오고 하자 하나도 없고 깔끔했어요 뽁뽁이로 포장도 해주셔서 너무 감사...,(긍정 확률 : 0.99) 긍정적인 리뷰입니다.,1,"[감사했어요, 같고, 거리는게, 걸리적, 그리고, 깔끔했어요, 나더라구요, 너무, ...","[감사했어요, 같고, 거리는게, 걸리적, 그리고, 깔끔했어요, 나더라구요, 너무, ..."
...,...,...,...,...,...,...,...
3601,에어팟 2세대,착용감,생각 보다 착용감이 좋습니다. 즐거운 사운드를 느꼈습니다,(긍정 확률 : 0.99) 긍정적인 리뷰입니다.,1,"[느꼈습니다, 보다, 사운드를, 생각, 좋습니다, 즐거운, 착용감이]","[느꼈습니다, 보다, 사운드를, 생각, 좋습니다, 즐거운, 착용감이]"
3602,에어팟 2세대,착용감,우선 이전에 에어팟을 써본적은 없고 갤럭시 기어 아이콘 2018년 형을 썼는데 에어...,(긍정 확률 : 0.99) 긍정적인 리뷰입니다.,1,"[2018년, 갤럭시, 귀가, 기어, 끼고, 사용이, 써본적은, 썼는데, 쓸때는, ...","[2018년, 갤럭시, 귀가, 기어, 끼고, 사용이, 써본적은, 썼는데, 쓸때는, ..."
3603,에어팟 2세대,휴대성,"휴대하기 편하고, 애플 기기 사이의 호환성이 좋은 무선 이어폰이 필요해서 에어팟을 ...",(긍정 확률 : 0.86) 긍정적인 리뷰입니다.,1,"[가지고, 가품인가, 간단한, 같습니다, 같은, 거부감이, 게다가, 계시고, 계신,...","[가지고, 가품인가, 간단한, 같습니다, 같은, 거부감이, 게다가, 계시고, 계신,..."
3604,에어팟 2세대,휴대성,휴대하기도 좋고 음질도 좋은데 떨어져서 잊어 버릴까 불안 하네요~,(긍정 확률 : 0.99) 긍정적인 리뷰입니다.,1,"[떨어져서, 버릴까, 불안, 음질도, 잊어, 좋고, 좋은데, 하네요, 휴대하기도]","[떨어져서, 버릴까, 불안, 음질도, 잊어, 좋고, 좋은데, 하네요, 휴대하기도]"


In [20]:
df['review_count'][0]

['2세대로',
 '귀가',
 '그리고',
 '깨끗하고',
 '넘어가요',
 '다음곡으로',
 '되게',
 '받았고',
 '받았습니다',
 '배송',
 '사용해보니',
 '실리콘케이스도',
 '안아프고',
 '알겠더라구요',
 '에어팟하는지',
 '역시',
 '음질이',
 '좋고',
 '좋아요_',
 '착용감이',
 '철가루랑',
 '톡톡',
 '편해요예쁘고',
 '하면',
 '하얀색']

In [21]:
df['review_tfidf'][0]

['2세대로',
 '귀가',
 '그리고',
 '깨끗하고',
 '넘어가요',
 '다음곡으로',
 '되게',
 '받았고',
 '받았습니다',
 '배송',
 '사용해보니',
 '실리콘케이스도',
 '안아프고',
 '알겠더라구요',
 '에어팟하는지',
 '역시',
 '음질이',
 '좋고',
 '좋아요_',
 '착용감이',
 '철가루랑',
 '톡톡',
 '편해요예쁘고',
 '하면',
 '하얀색']

단순히 문장을 키워드로 나누는 작업은 countvectorizer와 tfidfvectorizer 결과 동일

n_gram_range은 후보의 크기를 변경하는 데 사용할 수 있다 . 예를 들어 (3, 3)으로 설정하면 결과 후보는 키워드 3 개 를 포함하는 구문이 된다 .

그런 다음 변수 token_review는 후보 키워드 / 구를 포함하는 문자열 목록이다.


In [22]:
# 정리
df = df.drop(['review_tfidf'], axis=1)
df

,type,category,review,evaluation,label,review_count
0,에어팟 1세대,음질,2세대로 배송 받았고 철가루랑 하얀색 실리콘케이스도 잘 받았습니다 역시 사용해보니 ...,(긍정 확률 : 0.99) 긍정적인 리뷰입니다.,1,"[2세대로, 귀가, 그리고, 깨끗하고, 넘어가요, 다음곡으로, 되게, 받았고, 받았..."
1,에어팟 1세대,음질,"배송은 4시 이전 주문 당일 발송으로 다음날 바로 받았구요, 바로 정품 등록했는데 ...",(긍정 확률 : 0.99) 긍정적인 리뷰입니다.,1,"[4시, lpoint랑, 감사합니다, 구입해둔, 구입했습니다좋은, 다음날, 당일, ..."
2,에어팟 1세대,음질,아이폰과 아이패드를 둘 다 사용하고 있는데 아이폰에서 음악을 듣거나 영상을 보다 아...,(긍정 확률 : 0.99) 긍정적인 리뷰입니다.,1,"[2018년, 2일만에, 5월, 감싸고, 검색하다, 공휴일이, 구매자들, 구매하였습..."
3,에어팟 1세대,음질,다른 저렴한 곳도 있었습니다.\n\n그래도 제조일자 최신에 페이백으로 다시 받는 금...,(긍정 확률 : 0.99) 긍정적인 리뷰입니다.,1,"[가끔씩, 곳도, 괜찮다고, 구매해서, 구매했습니다, 귀에, 그래도, 금액, 꼽고,..."
4,에어팟 1세대,음질,택배도 하루일찍오고 하자 하나도 없고 깔끔했어요 뽁뽁이로 포장도 해주셔서 너무 감사...,(긍정 확률 : 0.99) 긍정적인 리뷰입니다.,1,"[감사했어요, 같고, 거리는게, 걸리적, 그리고, 깔끔했어요, 나더라구요, 너무, ..."
...,...,...,...,...,...,...
3601,에어팟 2세대,착용감,생각 보다 착용감이 좋습니다. 즐거운 사운드를 느꼈습니다,(긍정 확률 : 0.99) 긍정적인 리뷰입니다.,1,"[느꼈습니다, 보다, 사운드를, 생각, 좋습니다, 즐거운, 착용감이]"
3602,에어팟 2세대,착용감,우선 이전에 에어팟을 써본적은 없고 갤럭시 기어 아이콘 2018년 형을 썼는데 에어...,(긍정 확률 : 0.99) 긍정적인 리뷰입니다.,1,"[2018년, 갤럭시, 귀가, 기어, 끼고, 사용이, 써본적은, 썼는데, 쓸때는, ..."
3603,에어팟 2세대,휴대성,"휴대하기 편하고, 애플 기기 사이의 호환성이 좋은 무선 이어폰이 필요해서 에어팟을 ...",(긍정 확률 : 0.86) 긍정적인 리뷰입니다.,1,"[가지고, 가품인가, 간단한, 같습니다, 같은, 거부감이, 게다가, 계시고, 계신,..."
3604,에어팟 2세대,휴대성,휴대하기도 좋고 음질도 좋은데 떨어져서 잊어 버릴까 불안 하네요~,(긍정 확률 : 0.99) 긍정적인 리뷰입니다.,1,"[떨어져서, 버릴까, 불안, 음질도, 잊어, 좋고, 좋은데, 하네요, 휴대하기도]"


###4. 임베딩  
문서와 후보 키워드를 숫자 데이터로 변환한다. 


In [23]:
# 키워드 임베딩 모델 생성
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('distiluse-base-multilingual-cased-v2')

100%|██████████| 504M/504M [00:19<00:00, 26.5MB/s]


In [24]:
# 키워드 임베딩

review_embedded_list = []
count_embedded_list = []

for i in range(0, a):
  embedding_review = model.encode([review[i]])
  review_embedded_list.append(embedding_review)

for i in df['review_count']:
  embedding_count = model.encode(i)
  count_embedded_list.append(embedding_count)

In [25]:
len(review_embedded_list)

3606

In [26]:
len(count_embedded_list)

3606

변환기 모델에는 토큰 제한이 있으므로 큰 문서를 입력 할 때 몇 가지 오류가 발생할 수 있다. 이 경우 문서를 단락으로 분할하고 결과 벡터를 풀링 (평균 계산)하는 것을 고려할 수 있다.


###5. 코사인 유사성  
문서와 가장 유사한 후보가 문서를 나타내는 데 좋은 키워드 / 키 프레이즈라고 가정합니다.

임베딩된 키워드들은 전체 문장과 키워드 단위로 분리된 벡터 사이의 거리를 이용하여 코사인유사도를 계산한다. 원래 문장 벡터와 키워드의 벡터가 유사할수록 전체 문장을 잘 표현하는 단어라고 할 수 있다. 계산결과에서 가장 유사한 순서대로 5개의 키워드만 추출한다

In [27]:
from sklearn.metrics.pairwise import cosine_similarity

top_n = 5

In [28]:
a = len(review_embedded_list)
distances = []
keywords = []

for i in range(0,a):
  distance = cosine_similarity(review_embedded_list[i], count_embedded_list[i])
  keyword = [df['review_count'][i][index] for index in distance.argsort()[0][-top_n:]]

  distances.append(distance)
  keywords.append(keyword)

In [29]:
df

,type,category,review,evaluation,label,review_count
0,에어팟 1세대,음질,2세대로 배송 받았고 철가루랑 하얀색 실리콘케이스도 잘 받았습니다 역시 사용해보니 ...,(긍정 확률 : 0.99) 긍정적인 리뷰입니다.,1,"[2세대로, 귀가, 그리고, 깨끗하고, 넘어가요, 다음곡으로, 되게, 받았고, 받았..."
1,에어팟 1세대,음질,"배송은 4시 이전 주문 당일 발송으로 다음날 바로 받았구요, 바로 정품 등록했는데 ...",(긍정 확률 : 0.99) 긍정적인 리뷰입니다.,1,"[4시, lpoint랑, 감사합니다, 구입해둔, 구입했습니다좋은, 다음날, 당일, ..."
2,에어팟 1세대,음질,아이폰과 아이패드를 둘 다 사용하고 있는데 아이폰에서 음악을 듣거나 영상을 보다 아...,(긍정 확률 : 0.99) 긍정적인 리뷰입니다.,1,"[2018년, 2일만에, 5월, 감싸고, 검색하다, 공휴일이, 구매자들, 구매하였습..."
3,에어팟 1세대,음질,다른 저렴한 곳도 있었습니다.\n\n그래도 제조일자 최신에 페이백으로 다시 받는 금...,(긍정 확률 : 0.99) 긍정적인 리뷰입니다.,1,"[가끔씩, 곳도, 괜찮다고, 구매해서, 구매했습니다, 귀에, 그래도, 금액, 꼽고,..."
4,에어팟 1세대,음질,택배도 하루일찍오고 하자 하나도 없고 깔끔했어요 뽁뽁이로 포장도 해주셔서 너무 감사...,(긍정 확률 : 0.99) 긍정적인 리뷰입니다.,1,"[감사했어요, 같고, 거리는게, 걸리적, 그리고, 깔끔했어요, 나더라구요, 너무, ..."
...,...,...,...,...,...,...
3601,에어팟 2세대,착용감,생각 보다 착용감이 좋습니다. 즐거운 사운드를 느꼈습니다,(긍정 확률 : 0.99) 긍정적인 리뷰입니다.,1,"[느꼈습니다, 보다, 사운드를, 생각, 좋습니다, 즐거운, 착용감이]"
3602,에어팟 2세대,착용감,우선 이전에 에어팟을 써본적은 없고 갤럭시 기어 아이콘 2018년 형을 썼는데 에어...,(긍정 확률 : 0.99) 긍정적인 리뷰입니다.,1,"[2018년, 갤럭시, 귀가, 기어, 끼고, 사용이, 써본적은, 썼는데, 쓸때는, ..."
3603,에어팟 2세대,휴대성,"휴대하기 편하고, 애플 기기 사이의 호환성이 좋은 무선 이어폰이 필요해서 에어팟을 ...",(긍정 확률 : 0.86) 긍정적인 리뷰입니다.,1,"[가지고, 가품인가, 간단한, 같습니다, 같은, 거부감이, 게다가, 계시고, 계신,..."
3604,에어팟 2세대,휴대성,휴대하기도 좋고 음질도 좋은데 떨어져서 잊어 버릴까 불안 하네요~,(긍정 확률 : 0.99) 긍정적인 리뷰입니다.,1,"[떨어져서, 버릴까, 불안, 음질도, 잊어, 좋고, 좋은데, 하네요, 휴대하기도]"


In [30]:
distances[0].argsort()


array([[21,  4,  9, 12, 19, 15, 23,  5, 20,  2, 11,  6,  3,  0, 17, 24,
        13,  8,  7, 18, 16, 10, 14, 22,  1]])

In [31]:
distances[0].argsort()[0][-top_n:]

array([16, 10, 14, 22,  1])

In [32]:
keywords[0]

['음질이', '사용해보니', '에어팟하는지', '편해요예쁘고', '귀가']

그리고… 그게 다야! 입력 문서와 가장 유사한 상위 5 개 후보를 결과 키워드로 선택합니다.

In [33]:
df['keyword'] = keywords

In [34]:
df

,type,category,review,evaluation,label,review_count,keyword
0,에어팟 1세대,음질,2세대로 배송 받았고 철가루랑 하얀색 실리콘케이스도 잘 받았습니다 역시 사용해보니 ...,(긍정 확률 : 0.99) 긍정적인 리뷰입니다.,1,"[2세대로, 귀가, 그리고, 깨끗하고, 넘어가요, 다음곡으로, 되게, 받았고, 받았...","[음질이, 사용해보니, 에어팟하는지, 편해요예쁘고, 귀가]"
1,에어팟 1세대,음질,"배송은 4시 이전 주문 당일 발송으로 다음날 바로 받았구요, 바로 정품 등록했는데 ...",(긍정 확률 : 0.99) 긍정적인 리뷰입니다.,1,"[4시, lpoint랑, 감사합니다, 구입해둔, 구입했습니다좋은, 다음날, 당일, ...","[사용했어요가격도, 저렴하게, 등록했는데, 구입해둔, 구입했습니다좋은]"
2,에어팟 1세대,음질,아이폰과 아이패드를 둘 다 사용하고 있는데 아이폰에서 음악을 듣거나 영상을 보다 아...,(긍정 확률 : 0.99) 긍정적인 리뷰입니다.,1,"[2018년, 2일만에, 5월, 감싸고, 검색하다, 공휴일이, 구매자들, 구매하였습...","[이어폰을, 애플의, 아이폰에, 아이폰에서, 아이폰과]"
3,에어팟 1세대,음질,다른 저렴한 곳도 있었습니다.\n\n그래도 제조일자 최신에 페이백으로 다시 받는 금...,(긍정 확률 : 0.99) 긍정적인 리뷰입니다.,1,"[가끔씩, 곳도, 괜찮다고, 구매해서, 구매했습니다, 귀에, 그래도, 금액, 꼽고,...","[음질도, 귀에, 저렴한, 구매했습니다, 구매해서]"
4,에어팟 1세대,음질,택배도 하루일찍오고 하자 하나도 없고 깔끔했어요 뽁뽁이로 포장도 해주셔서 너무 감사...,(긍정 확률 : 0.99) 긍정적인 리뷰입니다.,1,"[감사했어요, 같고, 거리는게, 걸리적, 그리고, 깔끔했어요, 나더라구요, 너무, ...","[해주셔서, 하루일찍오고, 노래들으면서, 어제, 감사했어요]"
...,...,...,...,...,...,...,...
3601,에어팟 2세대,착용감,생각 보다 착용감이 좋습니다. 즐거운 사운드를 느꼈습니다,(긍정 확률 : 0.99) 긍정적인 리뷰입니다.,1,"[느꼈습니다, 보다, 사운드를, 생각, 좋습니다, 즐거운, 착용감이]","[착용감이, 생각, 즐거운, 사운드를, 느꼈습니다]"
3602,에어팟 2세대,착용감,우선 이전에 에어팟을 써본적은 없고 갤럭시 기어 아이콘 2018년 형을 썼는데 에어...,(긍정 확률 : 0.99) 긍정적인 리뷰입니다.,1,"[2018년, 갤럭시, 귀가, 기어, 끼고, 사용이, 써본적은, 썼는데, 쓸때는, ...","[통증이, 아이콘, 에어팟을, 에어팟은, 에어팟이]"
3603,에어팟 2세대,휴대성,"휴대하기 편하고, 애플 기기 사이의 호환성이 좋은 무선 이어폰이 필요해서 에어팟을 ...",(긍정 확률 : 0.86) 긍정적인 리뷰입니다.,1,"[가지고, 가품인가, 간단한, 같습니다, 같은, 거부감이, 게다가, 계시고, 계신,...","[이어폰을, 이어폰이, 이어폰이라서, 이어폰만, 아이폰]"
3604,에어팟 2세대,휴대성,휴대하기도 좋고 음질도 좋은데 떨어져서 잊어 버릴까 불안 하네요~,(긍정 확률 : 0.99) 긍정적인 리뷰입니다.,1,"[떨어져서, 버릴까, 불안, 음질도, 잊어, 좋고, 좋은데, 하네요, 휴대하기도]","[좋은데, 버릴까, 잊어, 휴대하기도, 음질도]"
